# Artificial Neural Network Model

### IF3170 - Machine Learning

Developed by: 
1. Juan Christopher Santoso (13521116)
2. Nicholas Liem (13521135)
3. Nathania Calista Djunaedi (13521139)
4. Antonio Natthan Krishna (13521162)

#### **Import Library**

In [1660]:
import numpy as np
import json
import copy
import csv

#### **JsonModelParser**

In [1661]:
class JsonModelParser:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = self.load_json_file()
        self.parse_model_data()

    def printDetails(self):
        print("\tINPUT SIZE:",self.input_size)
        print("\tLAYERS:", self.layers)
        print("\tINPUT:", self.input)
        print("\tINITIAL WEIGHTS:", self.initial_weights)
        print("\tTARGET:", self.target)
        print("\tLEARNING RATE:", self.learning_rate)
        print("\tBATCH SIZE:", self.batch_size)
        print("\tMAX ITERATION:", self.max_iteration)
        print("\tERROR THRESHOLD:", self.error_threshold)
    
    def load_json_file(self):
        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            print(f"The file {self.filepath} was not found")
            return None
        except json.JSONDecodeError:
            print(f"Error decoding JSON from the file {self.filepath}")
            return None

    def parse_model_data(self):
        if self.data:
            self.case = self.data.get('case', {})
            self.model = self.case.get('model', {})
            self.input_size = self.model.get('input_size')

            raw_layers = self.model.get('layers', [])   
            self.layers = [{'number_of_neurons': layer.get('number_of_neurons'),
                        'activation_function': layer.get('activation_function')}
                       for layer in raw_layers]
            
            self.input = self.case.get('input', [])
            self.initial_weights = self.case.get('initial_weights', [])
            self.target = self.case.get('target', [])
            self.parameters = self.case.get('learning_parameters', {})

            self.learning_rate = self.parameters.get('learning_rate')
            self.batch_size = self.parameters.get('batch_size')
            self.max_iteration = self.parameters.get('max_iteration')
            self.error_threshold = self.parameters.get('error_threshold')

            self.expect = self.data.get('expect', {})
            self.stopped_by = self.expect.get('stopped_by', '')
            self.final_weights = self.expect.get('final_weights', [])

#### **Neural Network Properties**

#### 1. Activation Function

In [1662]:
class ActivationFunction:
    def __init__(self, types='Sigmoid'):
        self.func = self.sigmoid
        self.dfunc = self.dsigmoid
        self.dfuncerr = self.dsum_square

        match types:
            case 'sigmoid':
                self.func = self.sigmoid
                self.dfunc = self.dsigmoid
                self.dfuncerr = self.dsum_square
            case 'linear':
                self.func = self.linear
                self.dfunc = self.dlinear
                self.dfuncerr = self.dsum_square
            case 'softmax':
                self.func = self.softmax
                # self.dfuncerr = self.derr_softmax
                self.dfuncerr = self.dsum_square
            case 'relu':
                self.func = self.relu
                self.dfunc = self.drelu
                self.dfuncerr = self.dsum_square

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def dsigmoid(self, x):
        sig = self.sigmoid(x)
        return sig * (1-sig)
    
    def linear(self, x):
        return x
    
    def dlinear(self, x):
        return 1
    
    def softmax(self, x):
        expX = np.exp(x - np.max(x, axis=1, keepdims=True))
        return expX / np.sum(expX, axis=1, keepdims=True)
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def drelu(self, x):
        return np.where(x > 0, 1, 0)
    
    def dsum_square(self, output, target):
        return target - output
    
    def derr_softmax(self, output, target):
        if (target != 1):
            return output
        else:
            return output - 1

#### 2. Loss Function

In [1663]:
class LossFunction:
    @staticmethod
    def calculate(output, target, layers):
        activation_mode = layers[-1]['activation_function']
        if (activation_mode == "softmax"):
            return LossFunction.loss_softmax(output, target)
        else:
            return LossFunction.loss_rsl(output, target)

    @staticmethod
    def loss_rsl(output, target):
        err = 0
        for i in range (len(output)):
            err += (target[i] - output[i])**2
        return 0.5 * err
    

    @staticmethod
    def loss_softmax(output, target):
        # idx = np.argmax(target)
        # return -1* np.log10(output[idx])

        err = 0
        for i in range(len(output)):
            err += -1* target[i] * np.log(output[i])
        return err
        

#### 3. Forward Propagation (Using Fast Forward Neural Network)

In [1664]:
class ForwardPropagation:
    @staticmethod
    def process(input_data, layers, weights):
        activations = input_data
        neuron_net = []
        neuron_out = []

        for i in range(len(layers)):
            activations_with_bias = np.insert(activations, 0, 1, axis=1)
            net_input = np.dot(activations_with_bias, weights[i])
            activation_mode = layers[i]['activation_function']
            activationFunc = ActivationFunction(activation_mode)
            activations = activationFunc.func(net_input)
            neuron_net.append(net_input)
            neuron_out.append(activations)

        return activations, neuron_net, neuron_out

#### 4. Backward Propagation

In [1665]:
class BackwardPropagation:
    @staticmethod
    def process(weights, output, target, neuron_out, layers, learning_rate, input_data):
        
        # Initiate variables
        delta = []
        delta_layer = []

        # Delta Output Layer
        activation_mode = layers[-1]['activation_function']
        activationFunc = ActivationFunction(activation_mode)

        # Handle Case for Softmax
        if (activation_mode == 'softmax'):
            for i in range (len(output)): 
                delta_layer.append(activationFunc.dfuncerr(output[i], target[i]))
        else:
            for i in range (len(output)): 
                delta_layer.append(activationFunc.dfuncerr(output[i], target[i]) * activationFunc.dfunc(output[i]))
        delta.append(delta_layer)

        # Delta Hidden Layer
        for i in range(len(layers) - 2, -1, -1):
            activation_mode = layers[i]['activation_function']
            activationFunc = ActivationFunction(activation_mode)

            # Save previous delta layer
            # If 2nd last, then return the Output layer
            # Else, return the previous layer but exclude the Bias val
            prev_delta_layer = delta_layer if (i == (len(layers)-2)) else delta_layer[1:]

            # Initiate new Delta Layer
            delta_layer = []
            layer_weight = weights[i+1]
            layer_output = neuron_out[i][0]

            # Iterate each neuron
            for j in range (len(layer_weight)): 
                neuron_weight = layer_weight[j]
                sigma = np.dot(neuron_weight, prev_delta_layer)

                # Make sure hidden layers are not softmax
                assert activation_mode != "softmax", "Softmax cannot be in hidden layers"
                delta_layer.append(sigma * activationFunc.dfunc(1 if j == 0 else layer_output[j-1]))
            # Append but push from front
            delta = [delta_layer[1:]] + delta

        # Update Weight
        # Iterate all the layers
        for i in range (len(layers)):
            if (i == 0):
                layer_input = input_data
            else :
                layer_input = neuron_out[i-1][0] # [0] because it is a list inside a list
            layer_input = np.insert(layer_input, 0, 1)
            
            # Iterate all weights
            for j in range (len(weights[i])):
                # Iterate all weights
                for k in range (len(weights[i][j])):
                    # It is based on formula wji = wji + learn_rate * dE/dnet * dnet/dw
                    weights[i][j][k] += learning_rate * (delta[i][k] * layer_input[j])
        
        return weights

#### **Artificial Neural Network**

In [1666]:
class ArtificialNeuralNetwork:
    def __init__(self, architecture = None):
      if architecture != None:
        self.layers = architecture.layers
        self.input_size = architecture.input_size
        self.learning_rate = architecture.learning_rate
        self.error_threshold = architecture.error_threshold
        self.max_iter = architecture.max_iteration
        self.batch_size = architecture.batch_size
        self.input_data = architecture.input
        self.target = architecture.target
        self.weights = architecture.initial_weights
        self.expected_stopped_by = architecture.stopped_by
        self.expect_weights = architecture.final_weights

    def predict(self):
        res = []
        for i in range (len(self.input_data)):
            output, _, _ = ForwardPropagation.process([self.input_data[i]], self.layers, self.weights)
            res.append(output)
        return res

    def train(self):
        temp_weight = copy.deepcopy(self.weights)

        for j in range (self.max_iter):
            # Set initial error_total. Reset every iteration
            error_total = 0
            minibatch = self.batch_size

            # batches_amount = math.ceil(len(self.input_data)/ minibatch)

            for i in range (len(self.input_data)):
                if (minibatch == 0):
                    self.weights = copy.deepcopy(temp_weight)
                    minibatch = self.batch_size
                # Forward Propagation
                output, _, neuron_out = ForwardPropagation.process([self.input_data[i]], self.layers, self.weights)
                # Calculate Loss
                error_total += LossFunction.calculate(output[0], self.target[i], self.layers)
                # Backward Propagation
                temp_weight = BackwardPropagation.process(temp_weight, output[0], self.target[i], neuron_out, self.layers, self.learning_rate, self.input_data[i])
                minibatch = minibatch - 1
            
            self.weights = copy.deepcopy(temp_weight)

            # Adjust error to mean value
            error_total /= len(self.input_data)

            if (error_total < self.error_threshold):
                break        

        if (j == self.max_iter-1):
            self.stopped_by = "MAX ITERATION"
        elif (error_total < self.error_threshold):
            self.stopped_by = "ERROR THRESHOLD"
        else:
            self.stopped_by = "UNIDENTIFIED"

        print("\n")
        print("LAST ITERATION :", j)
        print("TOTAL ERROR VAL:", error_total)
        print("\n")
        
        print("EXCPECTED")
        print("STOPPED BY :", self.expected_stopped_by)
        for weight_group in self.expect_weights:
            print("[")
            for weight in weight_group:
                print("\t", weight)
            print("]")
        print("\n")

        print("RESULT")
        print("STOPPED BY :", self.stopped_by)
        for weight_group in self.weights:
            print("[")
            for weight in weight_group:
                print("\t", weight)
            print("]")
    
    def save_model(self, file_path):
      data = {
          "case": {
              "model": {
                  "input_size": self.input_size,
                  "layers": self.layers
              },
              "initial_weights": self.weights,
              "learning_parameters": {
                  "learning_rate": self.learning_rate,
                  "batch_size": self.batch_size,
                  "max_iteration": self.max_iter,
                  "error_threshold": self.error_threshold
              }
          }
      }
      with open(file_path, 'w') as json_file:
          json.dump(data, json_file, indent=4)
  
    def load_model(self, file_path):
      architecture = JsonModelParser(file_path)
      self.layers = architecture.layers
      self.input_size = architecture.input_size
      self.learning_rate = architecture.learning_rate
      self.error_threshold = architecture.error_threshold
      self.max_iter = architecture.max_iteration
      self.batch_size = architecture.batch_size
      self.weights = architecture.initial_weights
      self.expected_stopped_by = architecture.stopped_by
      self.expect_weights = architecture.final_weights


#### **Preprocessing Data**

#### **Processing Data**

In [1667]:
def sum_squared_error(weights1, weights2):
    sse = 0
    for layer1, layer2 in zip(weights1, weights2):
        for neuron_weights1, neuron_weights2 in zip(layer1, layer2):
            for weight1, weight2 in zip(neuron_weights1, neuron_weights2):
                diff = weight1 - weight2
                sse += diff * diff
    return sse

In [1668]:

# Read all files inside the test folders
import os

currDir = os.getcwd()
testDir = currDir.replace("src", "test")
modelsDir = currDir.replace("src", "opt")

testFiles = os.listdir(testDir)
testFilesArr = []

for filename in testFiles:
  if (".json" in filename):
    testFilesArr.append(os.path.join(testDir, filename))


# Use every file inside testFilesArr as input

for testFile in testFilesArr:
  filename = os.path.basename(testFile)
  print("FILENAME:", filename)
  architecture = JsonModelParser(testFile)
  architecture.printDetails()
  print("\n")
  model = ArtificialNeuralNetwork(architecture)
  model.train()
  model.save_model(os.path.join(modelsDir, filename))

  error = sum_squared_error(model.weights, architecture.final_weights)

  print("\n")
  print("SSE (SUM SQUARED ERROR):", error)
  print("====================================================================")
  print("====================================================================")
  print("\n")

FILENAME: softmax_two_layer.json
	INPUT SIZE: 2
	LAYERS: [{'number_of_neurons': 4, 'activation_function': 'relu'}, {'number_of_neurons': 2, 'activation_function': 'softmax'}]
	INPUT: [[3.99, 2.96], [-0.71, 2.8], [-2.43, -0.2], [-1.9, 2.62], [-2.58, 1.43], [-3.43, -0.25], [1.15, -2.3], [4.28, 3.45]]
	INITIAL WEIGHTS: [[[0.1, -0.1, 0.1, -0.1], [-0.1, 0.1, -0.1, 0.1], [0.1, 0.1, -0.1, -0.1]], [[0.12, -0.1], [-0.12, 0.1], [0.12, -0.1], [-0.12, 0.1], [0.02, 0.0]]]
	TARGET: [[0, 1], [1, 0], [0, 1], [1, 0], [1, 0], [0, 1], [1, 0], [0, 1]]
	LEARNING RATE: 0.1
	BATCH SIZE: 1
	MAX ITERATION: 200
	ERROR THRESHOLD: 0.01




LAST ITERATION : 106
TOTAL ERROR VAL: 0.009936242476110826


EXCPECTED
STOPPED BY : error_threshold
[
	 [-0.28730211, -0.28822282, -0.70597451, 0.42094471]
	 [-0.5790794, -1.1836444, -1.34287961, 0.69575311]
	 [-0.41434377, 1.51314676, -0.97649086, -1.3043465]
]
[
	 [-1.72078607, 1.74078607]
	 [-0.50352956, 0.48352956]
	 [1.25764816, -1.23764816]
	 [-1.16998784, 1.14998784]
	 [



LAST ITERATION : 106
TOTAL ERROR VAL: 0.009936242476110826


EXCPECTED
STOPPED BY : error_threshold
[
	 [-0.28730211, -0.28822282, -0.70597451, 0.42094471]
	 [-0.5790794, -1.1836444, -1.34287961, 0.69575311]
	 [-0.41434377, 1.51314676, -0.97649086, -1.3043465]
]
[
	 [-1.72078607, 1.74078607]
	 [-0.50352956, 0.48352956]
	 [1.25764816, -1.23764816]
	 [-1.16998784, 1.14998784]
	 [1.0907634, -1.0707634]
]


RESULT
STOPPED BY : ERROR THRESHOLD
[
	 [-0.28730210942847484, -0.2882228247672504, -0.7059745091124834, 0.42094470920655785]
	 [-0.5790793999252973, -1.1836444019669583, -1.3428796084622932, 0.6957531055595404]
	 [-0.41434376959867913, 1.5131467608357148, -0.9764908601424801, -1.3043464969311707]
]
[
	 [-1.7207860719146808, 1.7407860719146813]
	 [-0.5035295626116906, 0.48352956261169]
	 [1.2576481557452113, -1.237648155745209]
	 [-1.1699878437697833, 1.1499878437697815]
	 [1.0907633975375395, -1.0707633975375392]
]


SSE (SUM SQUARED ERROR): 1.5845505031256717e-16


FILENAME: relu_b.

## IRIS

In [1669]:
# IRIS

inputs = []
targets = []
with open("iris.csv", "r") as csvfile:
    # Gunakan csv.DictReader
    datareader = csv.DictReader(csvfile)

    # Tidak perlu next(datareader), DictReader sudah meng-handle header
    for row in datareader:
        # Sekarang row adalah dictionary, dan Anda bisa mengakses nilai menggunakan nama kolom
        current = [
            float(row["SepalLengthCm"]),
            float(row["SepalWidthCm"]),
            float(row["PetalLengthCm"]),
            float(row["PetalWidthCm"])
        ]
        species = row["Species"]
        if(species == "Iris-setosa"):
            target = [1,0,0]
        elif(species == "Iris-versicolor"):
            target = [0,1,0]
        elif(species == "Iris-virginica"):
            target = [0,0,1]
        inputs.append(current)
        targets.append(target)

In [1670]:
architecture = JsonModelParser("iris_config.json")
architecture.input_size = len(inputs[0])
architecture.input = inputs
architecture.target = targets
model = ArtificialNeuralNetwork(architecture)
model.train()
model.save_model(os.path.join(os.getcwd().replace("src", "opt"), "iris_train.json"))



LAST ITERATION : 999
TOTAL ERROR VAL: 0.10799645198590958


EXCPECTED
STOPPED BY : 


RESULT
STOPPED BY : MAX ITERATION
[
	 [0.587073476645592, 0.8980810347582467, -1.1851545114038078]
	 [1.1208595088687452, 0.9597290374574284, -1.7805885463261513]
	 [2.430619599353256, -0.028021295700770478, -2.102598303652427]
	 [-3.075217470038597, -0.22797076876503622, 3.6031882388036136]
	 [-1.383558506239567, -1.219106126141017, 2.9026646323805902]
]


In [1671]:
loaded_model = ArtificialNeuralNetwork(None)
loaded_model.load_model(os.path.join(modelsDir, "iris_train.json"))
loaded_model.input_data = inputs
loaded_model.target = targets
res_code = loaded_model.predict()
res = []
for i in range (len(res_code)):
    idmax = np.argmax(res_code[i])
    if idmax == 0:
        res.append('Iris-setosa')
    elif idmax == 1:
        res.append('Iris-versicolor')
    else:
        res.append('Iris-virginica')
res

['Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-setosa',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',


## Load Model Implementation

In [1672]:
currDir = os.getcwd()
testDir = currDir.replace("src", "test")
modelsDir = currDir.replace("src", "opt")

loaded_model = ArtificialNeuralNetwork(None)
loaded_model.load_model(os.path.join(modelsDir, "linear_small_lr.json"))